In [1]:
# 加载必要的包
from os import makedirs, path
import os
import numpy as np
import pandas as pd
from IPython.display import display
from nibabel import save
from nilearn import image, plotting, reporting
from nimare import correct, io, meta, utils #主要用nimare完成元分析
from scipy.stats import norm

# We are now ready to perform the actual ALE analyses with NiMARE. 
# We write a custom function which takes a single Sleuth text file as its input and (a) calculates the ALE map, 
# (b) corrects for multiple comparisons using a Monte Carlo-based FWE correction, 
# and (c) stores the cluster level-thresholded maps into the output directory. 
# We then apply this function to all the Sleuth files we have created in the previous step. 


In [6]:
# Define function for performing a single ALE analysis with FWE correction
def run_ale(text_file, voxel_thresh, cluster_thresh, random_seed, n_iters, output_dir):

    # Set a random seed to make the results reproducible
    if random_seed:
        np.random.seed(random_seed)

    # Perform the ALE
    dset = io.convert_sleuth_to_dataset(text_file=text_file, target="mni152_2mm")
    ale = meta.cbma.ALE()
    res = ale.fit(dset)

    # FWE correction for multiple comparisons
    corr = correct.FWECorrector(
        method="montecarlo", voxel_thresh=voxel_thresh, n_iters=n_iters
    )
    cres = corr.transform(result=res)

    # Save maps to the ouput directory
    prefix = path.basename(text_file).replace(".txt", "")
    res.save_maps(output_dir=output_dir, prefix=prefix)
    cres.save_maps(output_dir=output_dir, prefix=prefix)

    # Create cluster-level thresholded z and ALE maps
    img_clust_mass = cres.get_map("z_desc-mass_level-cluster_corr-FWE_method-montecarlo")
    img_clust_size = cres.get_map("z_desc-size_level-cluster_corr-FWE_method-montecarlo")
    img_z = cres.get_map("z")
    img_ale = cres.get_map("stat")
    cluster_thresh_z = norm.ppf(1 - cluster_thresh / 2)
    img_clust_mass_thresh = image.threshold_img(img=img_clust_mass, threshold=cluster_thresh_z)
    img_clust_size_thresh = image.threshold_img(img=img_clust_size, threshold=cluster_thresh_z)
    img_mask = image.math_img("np.where(img > 0, 1, 0)", img=img_clust_size_thresh)
    #img_z_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_z)
    img_ale_thresh = image.math_img("img1 * img2", img1=img_mask, img2=img_ale)

    # Save thresholded maps to the output directory
    save(img=img_clust_mass_thresh, filename=output_dir + "/" + prefix + "_z_mass_level_thresh.nii.gz")
    save(img=img_clust_size_thresh, filename=output_dir + "/" + prefix + "_z_size_level_thresh.nii.gz")
    # Save ALE thresholded for tables later on
    save(img=img_ale_thresh, filename=output_dir + "/" + prefix + "_stat_size_thresh.nii.gz")


# if __name__ == "__main__":

#     # Apply our function to all the Sleuth files
#         run_ale(
#             text_file='/Users/ss/Documents/Psych_ALE_meta/data/Self_all.txt','/Users/ss/Documents/Psych_ALE_meta/data/Self_st.txt'
#             voxel_thresh=0.001,
#             cluster_thresh=0.05,
#             random_seed=1234,
#             n_iters=10000,
#             output_dir="../results/ale/",
#         )

if __name__ == "__main__":

    # 定义一个包含所有文本文件路径的列表
    all_files = [
        # '/Users/ss/Documents/Psych_ALE_meta/data/Self_all.txt',
        # '/Users/ss/Documents/Psych_ALE_meta/data/Self_st.txt',
        '/Users/ss/Documents/Psych_ALE_meta/data/unhealth.txt',
        # '/Users/ss/Documents/Psych_ALE_meta/data/health.txt'
    ]

    # 循环遍历每个文件，依次运行 ALE 分析
    for text_file in all_files:
        run_ale(
            text_file=text_file,
            voxel_thresh=0.001,
            cluster_thresh=0.05,
            random_seed=1234,
            n_iters=5000,
            output_dir="../results/ale/"
        )


INFO:nimare.correct:Using correction method implemented in Estimator: nimare.meta.cbma.ale.ALE.correct_fwe_montecarlo.


  0%|          | 0/5000 [00:00<?, ?it/s]

INFO:nimare.meta.cbma.base:Using null distribution for voxel-level FWE correction.
/opt/anaconda3/lib/python3.12/site-packages/nilearn/image/image.py:1106: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)
